**Import Libraries**

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

**Import Data**

In [3]:
movies_data = pd.read_csv("movies.csv")
ratings_data = pd.read_csv("ratings.csv")

**A.** Show 5 First and Last Data

In [4]:
# A
print(movies_data.head())
print(movies_data.tail())
print(ratings_data.head())
print(ratings_data.tail())

print('movies data shape: ', np.shape(movies_data))
print('ratings data shape: ', np.shape(ratings_data))

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
      movieId                                      title  \
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                               genres  
9737

In [5]:
# A'
print([movies_data[0:5], movies_data[-5:]])
print([ratings_data[0:5], ratings_data[-5:]])

print('movies data shape: ', np.shape(movies_data))
print('ratings data shape: ', np.shape(ratings_data))

[   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  ,       movieId                                      title  \
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                               genres  
97

**B.** Merge Movies and Ratings Data

In [6]:
# B
merged_data = movies_data.merge(ratings_data, how = 'inner', on = 'movieId')
merged_data

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021


**C.** Delete Extra Columns

In [7]:
# C
merged_data.drop('title', inplace = True, axis = 1)
merged_data.drop('genres', inplace = True, axis = 1)
merged_data.drop('timestamp', inplace = True, axis = 1)
merged_data

,movieId,userId,rating
0,1,1,4.0
1,1,5,4.0
2,1,7,4.5
3,1,15,2.5
4,1,17,4.5
...,...,...,...
100831,193581,184,4.0
100832,193583,184,3.5
100833,193585,184,3.5
100834,193587,184,3.5


**D.** Group Data with GroupBy Method

In [8]:
# D
data = merged_data.groupby('userId', sort = True)
data.first()

,movieId,rating
userId,,
1,1,4.0
2,318,3.0
3,31,0.5
4,21,3.0
5,1,4.0
...,...,...
606,1,2.5
607,1,4.0
608,1,2.5


**E.** Normalized Users Ratings

In [9]:
# E
train_X = preprocessing.normalize([data['rating'].mean()])
train_X = torch.FloatTensor(train_X)

**F.** Create Neurons

In [31]:
# F
class RBM():
  def __init__(self, hidden_node, visible_node):
    self.weights = np.random.normal(0, 1, [hidden_node, visible_node])
    self.bias_hidden_give_visible = np.random.normal(0, 1, hidden_node)
    self.bias_visible_give_hidden = np.random.normal(0, 1, visible_node)

  def hidden_node_sample(self, x):
    weights_hidden_node = np.dot(x, np.transpose(self.weights))
    activation_function = weights_hidden_node + self.bias_hidden_give_visible*np.ones([np.shape(weights_hidden_node)[0], np.shape(weights_hidden_node)[1]])
    activation_function = torch.FloatTensor(activation_function)
    return torch.sigmoid(activation_function)

  def visible_node_sample(self, x):
    weights_visible_node = np.dot(x, self.weights)
    activation_function = weights_visible_node + self.bias_hidden_give_visible*np.ones([np.shape(weights_visible_node)[0], np.shape(weights_visible_node)[1]])
    activation_function = torch.FloatTensor(activation_function)
    return torch.sigmoid(activation_function)

  def train(self, v0, vk, ph0, phk):
    self.weights += np.dot(np.transpose(v0), ph0) - np.dot(np.transpose(vk), phk)
    self.bias_hidden_give_visible += np.sum((ph0 - phk), axis = 0)
    self.bias_visible_give_hidden += np.sum((v0 - vk), axis = 0)


**G.** Train Neuron

In [ ]:
# G


In [ ]:
from google.colab import drive
drive.mount('/content/drive')